In [ ]:
#| default_exp datastructure.team_stats

In [ ]:
from IPython.core.debugger import set_trace

%load_ext autoreload
%autoreload 2

#  Team Stats
> Data structure required to extract Instat Team Stats.

In [ ]:
#| export

import pandas as pd
import mongoengine
import datetime
import logging
from typing import List
from mongoengine.fields import ListField
from soccer_prediction_data.config.localconfig import CONFIG

## Action

The Action class provides all of the necessary information regarding the stats recorded during a given team's game.

Example of an Action is the number of shots. 

In [ ]:
#| export


class Action(mongoengine.EmbeddedDocument):
    "Stores main information about the reported stats of a given team in a given game."
    # Action Ids.
    action_id = mongoengine.IntField(db_field="actionId", required=True)
    # Action name.
    action_name = mongoengine.StringField(db_field="actionName", required=True)
    # Value.
    value = mongoengine.IntField(db_field="value", required=True)
    # Time Periods.
    time_periods = ListField(db_field="timePeriods", required=False)
    # Zones.
    zones = ListField(db_field="zones", required=False)

## Teams

These features are computed at the game level and contains:

+ **Metadata**: general information about game Id, a given team identity. 
+ **Stats** information: the entire set of metrics realised throughout the game.

In [ ]:
# | export


class TeamStats(mongoengine.Document):
    "Store the team stats for a given game identified by its id"

    # Game-id.
    game_id = mongoengine.IntField(db_field="gameId", required=True)

    # Team.
    team_id = mongoengine.IntField(db_field="teamId", required=True)
    team_name = mongoengine.StringField(db_field="teamName", required=True)

    # Stats.
    stats = mongoengine.EmbeddedDocumentListField(Action)

    meta = {
        "db_alias": "inStat",
        "collection": "teamStats",
    }

    @classmethod
    def get_all_teams_stats(
        cls,
        limit: int = None,  # Number of rows.
    ):  # List of TeamStats output.
        "Extract all documents."
        return cls.objects().limit(limit)

    @classmethod
    def get_game_team_stats(
        cls,
        game_id: int,  # Instat game identifier.
        team_id: int,  # Instat team identifier.
    ):  # Fixture output.
        "Extract a single game stats by its gameId and teamId."
        return cls.objects(game_id=game_id, team_id=team_id).first()

In [ ]:
from soccer_prediction_data.config.mongo import mongo_init

In [ ]:
# Initialise connections.
mongo_init("prod_atlas")
# GameId.
game_id = 1199088
# Team Id.
team_id = 1059
# Extract team stats during the game.
team_stats = TeamStats.get_game_team_stats(
    game_id=game_id,
    team_id=team_id,
)
first_stat = team_stats.stats.first()
{x:first_stat[x] for x in first_stat}

{'action_id': -18,
 'action_name': 'Goals scored - Result',
 'value': 1,
 'time_periods': [{'t4': 1.0}],
 'zones': [{'z4b': 1.0}]}

In [ ]:
#| hide

import nbdev

nbdev.nbdev_export()